# Load architecture

In [1]:
from wwv.config import Config 
MODEL_DIR = "/home/akinwilson/Code/pytorch/output/model"
DATA_DIR = "/home/akinwilson/Code/pytorch/dataset/keywords"
LR_RANGE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5][1]
BATCH_SIZE_RANGE = [1,2,32, 64, 128, 256][2]
EPOCH_RANGE = [1, 10, 30, 50, 100, 1000][1]
ES_PATIENCE_RANGE = [1, 10, 20, 100, 200][2]
MODELS = ["VecM5", "Resnet2vec1D","SpecResnet2D", "HSTAT", "DeepSpeech", "ResNet"][-3]
AUDIO_FEATURE_OPT = ["spectrogram", "mfcc", "pcm"][0]
PRETRAINED_MODEL_NAME_OR_PATH = "facebook/wav2vec2-base-960h"
AUGS = False
params = {
    "audio_duration":3,
    "sample_rate":16000,
    "model_name": MODELS,
    "verbose": False,
    "path": {
        "model_dir": MODEL_DIR,
        "data_dir": DATA_DIR,
        "pretrained_name_or_path": PRETRAINED_MODEL_NAME_OR_PATH
        },
    "fit_param": {"init_lr":LR_RANGE, "weight_decay":0.0001, "max_epochs":EPOCH_RANGE, "gamma": 0.1,"es_patience":ES_PATIENCE_RANGE}, 
    "data_param":{"train_batch_size": BATCH_SIZE_RANGE, "val_batch_size": BATCH_SIZE_RANGE,"test_batch_size": BATCH_SIZE_RANGE}, 
    "audio_feature": AUDIO_FEATURE_OPT,
    "audio_feature_param": { "mfcc":{"sr":16000,"n_mfcc":20,"norm": 'ortho',"verbose":True,"ref":1.0,"amin":1e-10,"top_db":80.0,"hop_length":512,},
                            "spectrogram":{"sr":16000, "n_fft":2048, "win_length":None,"n_mels":128,"hop_length":512,"window":'hann',"center":True,"pad_mode":'reflect',"power":2.0,"htk":False,"fmin":0.0,"fmax":None,"norm":1,"trainable_mel":False,"trainable_STFT":False,"verbose": True },
                            "pcm": {}},
    "augmentation":{'Gain': AUGS, 'PitchShift': AUGS, 'Shift': AUGS},
    "augmentation_param":{"Gain": {  "min_gain_in_db":-18.0,"max_gain_in_db":  6.0,"mode":'per_example',"p":1,"p_mode":'per_example'},
                        "PitchShift": {"min_transpose_semitones": -4.0, "max_transpose_semitones": 4.0,"mode":'per_example',"p":1,"p_mode":'per_example',"sample_rate":16000,"target_rate": None,"output_type": None,},
                        "Shift":{ "min_shift":-0.5,"max_shift": 0.5,"shift_unit":'fraction',"rollover": True,"mode":'per_example',"p":1,"p_mode": 'per_example',"sample_rate": 16000,"target_rate":None,"output_type":None}},
    }
cfg = Config(params)

## Transformer

#### Layers

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from itertools import repeat
import collections.abc
import math
import warnings
from torch.nn.init import _calculate_fan_in_and_fan_out
# from PyTorch internals
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return x
        return tuple(repeat(x, n))
    return parse

to_1tuple = _ntuple(1)
to_2tuple = _ntuple(2)
to_3tuple = _ntuple(3)
to_4tuple = _ntuple(4)
to_ntuple = _ntuple


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)

class PatchEmbed(nn.Module):
    """ 2D Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768, norm_layer=None, flatten=True, patch_stride = 16):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        patch_stride = to_2tuple(patch_stride)
        self.img_size = img_size
        self.patch_size = patch_size
        self.patch_stride = patch_stride
        self.grid_size = (img_size[0] // patch_stride[0], img_size[1] // patch_stride[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.flatten = flatten
        self.in_chans = in_chans
        self.embed_dim = embed_dim
        
        padding = ((patch_size[0] - patch_stride[0]) // 2, (patch_size[1] - patch_stride[1]) // 2)

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_stride, padding=padding)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x)
        if self.flatten:
            x = x.flatten(2).transpose(1, 2)  # BCHW -> BNC
        x = self.norm(x)
        return x

class Mlp(nn.Module):
    """ MLP as used in Vision Transformer, MLP-Mixer and related networks
    """
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)


def variance_scaling_(tensor, scale=1.0, mode='fan_in', distribution='normal'):
    fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
    if mode == 'fan_in':
        denom = fan_in
    elif mode == 'fan_out':
        denom = fan_out
    elif mode == 'fan_avg':
        denom = (fan_in + fan_out) / 2

    variance = scale / denom

    if distribution == "truncated_normal":
        # constant is stddev of standard normal truncated to (-2, 2)
        trunc_normal_(tensor, std=math.sqrt(variance) / .87962566103423978)
    elif distribution == "normal":
        tensor.normal_(std=math.sqrt(variance))
    elif distribution == "uniform":
        bound = math.sqrt(3 * variance)
        tensor.uniform_(-bound, bound)
    else:
        raise ValueError(f"invalid distribution {distribution}")


def lecun_normal_(tensor):
    variance_scaling_(tensor, mode='fan_in', distribution='truncated_normal')





/home/akinwilson/.local/share/virtualenvs/pytorch-T3BHxh3q/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Model

In [4]:
import logging
import pdb
import math
import random
from numpy.core.fromnumeric import clip, reshape
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint

# from torchlibrosa.stft import Spectrogram, LogmelFilterBank
# from torchlibrosa.augmentation import SpecAugmentation

from itertools import repeat
from typing import List
# from .layers import PatchEmbed, Mlp, DropPath, trunc_normal_, to_2tuple
# from utils import do_mixup, interpolate

    
def interpolate(x, ratio):
    """Interpolate data in time domain. This is used to compensate the 
    resolution reduction in downsampling of a CNN.
    
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate

    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled

def do_mixup(x, mixup_lambda):
    """
    Args:
      x: (batch_size , ...)
      mixup_lambda: (batch_size,)

    Returns:
      out: (batch_size, ...)
    """
    out = (x.transpose(0,-1) * mixup_lambda + torch.flip(x, dims = [0]).transpose(0,-1) * (1 - mixup_lambda)).transpose(0,-1)
    return out


# below codes are based and referred from https://github.com/microsoft/Swin-Transformer
# Swin Transformer for Computer Vision: https://arxiv.org/pdf/2103.14030.pdf

def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size
    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows


def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x


class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x, attn

    def extra_repr(self):
        return f'dim={self.dim}, window_size={self.window_size}, num_heads={self.num_heads}'


# We use the model based on Swintransformer Block, therefore we can use the swin-transformer pretrained model
class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm, norm_before_mlp='ln'):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        self.norm_before_mlp = norm_before_mlp
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        if self.norm_before_mlp == 'ln':
            self.norm2 = nn.LayerNorm(dim)
        elif self.norm_before_mlp == 'bn':
            self.norm2 = lambda x: nn.BatchNorm1d(dim)(x.transpose(1, 2)).transpose(1, 2)
        else:
            raise NotImplementedError
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        # pdb.set_trace()
        H, W = self.input_resolution
        # print("H: ", H)
        # print("W: ", W)
        # pdb.set_trace()
        B, L, C = x.shape
        # assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows, attn = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x, attn

    def extra_repr(self):
        return f"dim={self.dim}, input_resolution={self.input_resolution}, num_heads={self.num_heads}, " \
               f"window_size={self.window_size}, shift_size={self.shift_size}, mlp_ratio={self.mlp_ratio}"



class PatchMerging(nn.Module):
    r""" Patch Merging Layer.
    Args:
        input_resolution (tuple[int]): Resolution of input feature.
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = x.view(B, -1, 4 * C)  # B H/2*W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x

    def extra_repr(self):
        return f"input_resolution={self.input_resolution}, dim={self.dim}"


class BasicLayer(nn.Module):
    """ A basic Swin Transformer layer for one stage.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False.
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False,
                 norm_before_mlp='ln'):

        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim=dim, input_resolution=input_resolution,
                                 num_heads=num_heads, window_size=window_size,
                                 shift_size=0 if (i % 2 == 0) else window_size // 2,
                                 mlp_ratio=mlp_ratio,
                                 qkv_bias=qkv_bias, qk_scale=qk_scale,
                                 drop=drop, attn_drop=attn_drop,
                                 drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                                 norm_layer=norm_layer, norm_before_mlp=norm_before_mlp)
            for i in range(depth)])

        # patch merging layer
        if downsample is not None:
            self.downsample = downsample(input_resolution, dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x):
        attns = []
        for blk in self.blocks:
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x)
            else:
                x, attn = blk(x)
                if not self.training:
                    attns.append(attn.unsqueeze(0))
        if self.downsample is not None:
            x = self.downsample(x)
        if not self.training:
            attn = torch.cat(attns, dim = 0)
            attn = torch.mean(attn, dim = 0)
        return x, attn

    def extra_repr(self):
        return f"dim={self.dim}, input_resolution={self.input_resolution}, depth={self.depth}"


# The Core of HTSAT
class HTSAT_Swin_Transformer(nn.Module):
    r"""HTSAT based on the Swin Transformer
    Args:
        spec_size (int | tuple(int)): Input Spectrogram size. Default 256
        patch_size (int | tuple(int)): Patch size. Default: 4
        path_stride (iot | tuple(int)): Patch Stride for Frequency and Time Axis. Default: 4
        in_chans (int): Number of input image channels. Default: 1 (mono)
        num_classes (int): Number of classes for classification head. Default: 527
        embed_dim (int): Patch embedding dimension. Default: 96
        depths (tuple(int)): Depth of each HTSAT-Swin Transformer layer.
        num_heads (tuple(int)): Number of attention heads in different layers.
        window_size (int): Window size. Default: 8
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
        qkv_bias (bool): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float): Override default qk scale of head_dim ** -0.5 if set. Default: None
        drop_rate (float): Dropout rate. Default: 0
        attn_drop_rate (float): Attention dropout rate. Default: 0
        drop_path_rate (float): Stochastic depth rate. Default: 0.1
        norm_layer (nn.Module): Normalization layer. Default: nn.LayerNorm.
        ape (bool): If True, add absolute position embedding to the patch embedding. Default: False
        patch_norm (bool): If True, add normalization after patch embedding. Default: True
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False
        config (module): The configuration Module from config.py
    """

    def __init__(self,
     spec_size=128, patch_size=4, patch_stride=(4,4), 
                in_chans=1, num_classes=1,
                 embed_dim=96, depths=[2, 2, 6, 2], num_heads=[4, 8, 16, 32],
                 window_size=8, mlp_ratio=4., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm,  patch_norm=True,
                 ape=False,
                 use_checkpoint=False, norm_before_mlp='ln', config = None, **kwargs):
        super().__init__()

        self.config = config
        self.spec_size = spec_size 
        self.patch_stride = patch_stride
        self.patch_size = patch_size
        self.window_size = window_size
        self.embed_dim = embed_dim
        self.depths = depths
        self.ape = ape
        self.in_chans = in_chans
        self.num_classes = num_classes
        self.num_heads = num_heads
        self.num_layers = len(self.depths)
        self.num_features = int(self.embed_dim * 2 ** (self.num_layers - 1))
        
        self.drop_rate = drop_rate
        self.attn_drop_rate = attn_drop_rate
        self.drop_path_rate = drop_path_rate

        self.qkv_bias = qkv_bias
        self.qk_scale = None

        self.patch_norm = patch_norm
        self.norm_layer = norm_layer if self.patch_norm else None
        self.norm_before_mlp = norm_before_mlp
        self.mlp_ratio = mlp_ratio

        self.use_checkpoint = use_checkpoint

        #  process mel-spec ; used only once
        self.freq_ratio = self.spec_size // self.config.mel_bins
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None
        self.interpolate_ratio = 32     # Downsampled ratio
        # Spectrogram extractor
        # self.spectrogram_extractor = Spectrogram(n_fft=config.window_size, hop_length=config.hop_size, 
        #     win_length=config.window_size, window=window, center=center, pad_mode=pad_mode, 
        #     freeze_parameters=True)
        # Logmel feature extractor
        # self.logmel_extractor = LogmelFilterBank(sr=config.sample_rate, n_fft=config.window_size, 
        #     n_mels=config.mel_bins, fmin=config.fmin, fmax=config.fmax, ref=ref, amin=amin, top_db=top_db, 
        #     freeze_parameters=True)
        # Spec augmenter
        # self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
        #     freq_drop_width=8, freq_stripes_num=2) # 2 2

        MEL_BINS = 128
        self.bn0 = nn.BatchNorm2d(MEL_BINS)


        # split spctrogram into non-overlapping patches
        self.patch_embed = PatchEmbed(
            img_size=self.spec_size, patch_size=self.patch_size, in_chans=self.in_chans, 
            embed_dim=self.embed_dim, norm_layer=self.norm_layer, patch_stride = patch_stride)
        
        num_patches = self.patch_embed.num_patches
        patches_resolution = self.patch_embed.grid_size
        # print(f"num_patches: {num_patches}")
        self.patches_resolution = patches_resolution
        # print(f"patches_resolution: {patches_resolution}")

        # absolute position embedding
        # if self.ape:
        #     self.absolute_pos_embed = nn.Parameter(torch.zeros(1, num_patches, self.embed_dim))
        #     trunc_normal_(self.absolute_pos_embed, std=.02)

        self.pos_drop = nn.Dropout(p=self.drop_rate)

        # stochastic depth
        dpr = [x.item() for x in torch.linspace(0, self.drop_path_rate, sum(self.depths))]  # stochastic depth decay rule

        # build layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(dim=int(self.embed_dim * 2 ** i_layer),
                input_resolution=(patches_resolution[0] // (2 ** i_layer),
                                    patches_resolution[1] // (2 ** i_layer)),
                depth=self.depths[i_layer],
                num_heads=self.num_heads[i_layer],
                window_size=self.window_size,
                mlp_ratio=self.mlp_ratio,
                qkv_bias=self.qkv_bias, qk_scale=self.qk_scale,
                drop=self.drop_rate, attn_drop=self.attn_drop_rate,
                drop_path=dpr[sum(self.depths[:i_layer]):sum(self.depths[:i_layer + 1])],
                norm_layer=self.norm_layer,
                downsample=PatchMerging if (i_layer < self.num_layers - 1) else None,
                use_checkpoint=use_checkpoint,
                norm_before_mlp=self.norm_before_mlp)
            self.layers.append(layer)

        self.norm = self.norm_layer(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.maxpool = nn.AdaptiveMaxPool1d(1)
        self.head = nn.Linear(self.num_features, 1) 

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'absolute_pos_embed'}

    @torch.jit.ignore
    def no_weight_decay_keywords(self):
        return {'relative_position_bias_table'}


    def forward_features(self, x):
        # A deprecated optimization for using a hierarchical output from different blocks
        # if self.config.htsat_hier_output:
        #     hier_x = []
        #     hier_attn = []
        # print(f"forward_features(self, x): {x.shape}")
        # print(f"forward_features [in], {}")
        
        frames_num = x.shape[2]        
        # print(f"frames_num: {frames_num}")
        x = self.patch_embed(x)
        # print(f"forward_features[self.patch_embed(x)] [out]: {x.shape}")
        if self.ape:
            x = x + self.absolute_pos_embed
        
        x = self.pos_drop(x)
        # print(f"forward_features[self.pos_drop(x)] [out]: {x.shape}")
        for i, layer in enumerate(self.layers):
            x, attn = layer(x)

        print(f"forward_features[x, attn = layer(x)] [out]: {x.shape}")
        # print(f"forward_features[enumerate(self.layers)] [out]: {x.shape}")
        x = self.norm(x)  # B N C
        print(f"forward_features[self.norm(x)] [out]: {x.shape}")
        B, N, C = x.shape
        # orward_features[enumerate(self.layers)] [out]: torch.Size([11, 64, 768])
        
        fpx = x.permute(0,2,1).contiguous().reshape(B, C, frames_num // (2 ** (len(self.depths) + 1)), frames_num // (2 ** (len(self.depths) + 1)) )
        # print(f"fpx shape {fpx.shape}")

        B, C, F, T = fpx.shape

        # framewise 
        c_freq_bin = F // self.freq_ratio
        fpx = fpx.reshape(B, C, F // c_freq_bin, c_freq_bin, T)
        fpx = fpx.permute(0,1,3,2,4).contiguous().reshape(B, C, c_freq_bin, -1)
        fpx = torch.sum(fpx, dim = 2)
        fpx = interpolate(fpx.permute(0,2,1).contiguous(), 8 * self.patch_stride[1]) 
        # framewise 

        x = self.avgpool(x.transpose(1, 2))  # B C 1
        x = torch.flatten(x, 1)
        x = self.head(x)
        # fpx = self.head(fpx)

        output_dict = {
            'framewise_output': fpx, 
            'clipwise_output': x
            }
        return output_dict

    def crop_wav(self, x, crop_size, spe_pos = None):
        time_steps = x.shape[2]
        tx = torch.zeros(x.shape[0], x.shape[1], crop_size, x.shape[3]).to(x.device)
        for i in range(len(x)):
            if spe_pos is None:
                crop_pos = random.randint(0, time_steps - crop_size - 1)
            else:
                crop_pos = spe_pos
            tx[i][0] = x[i, 0, crop_pos:crop_pos + crop_size,:]
        return tx

    # Reshape the wavform to a img size, if you want to use the pretrained swin transformer model
    def reshape_wav2img(self, x):
        B, C, T, F = x.shape
        target_T = int(self.spec_size * self.freq_ratio)
        target_F = self.spec_size // self.freq_ratio
        assert T <= target_T and F <= target_F, "the wav size should less than or equal to the swin input size"
        # to avoid bicubic zero error
        if T < target_T:
            x = nn.functional.interpolate(x, (target_T, x.shape[3]), mode="bicubic", align_corners=True)
        if F < target_F:
            x = nn.functional.interpolate(x, (x.shape[2], target_F), mode="bicubic", align_corners=True)
        x = x.permute(0,1,3,2).contiguous()
        x = x.reshape(x.shape[0], x.shape[1], x.shape[2], self.freq_ratio, x.shape[3] // self.freq_ratio)
        # print(x.shape)
        x = x.permute(0,1,3,2,4).contiguous()
        x = x.reshape(x.shape[0], x.shape[1], x.shape[2] * x.shape[3], x.shape[4])
        return x
    
    # Repeat the wavform to a img size, if you want to use the pretrained swin transformer model
    def repeat_wat2img(self, x, cur_pos):
        B, C, T, F = x.shape
        target_T = int(self.spec_size * self.freq_ratio)
        target_F = self.spec_size // self.freq_ratio
        assert T <= target_T and F <= target_F, "the wav size should less than or equal to the swin input size"
        # to avoid bicubic zero error
        if T < target_T:
            x = nn.functional.interpolate(x, (target_T, x.shape[3]), mode="bicubic", align_corners=True)
        if F < target_F:
            x = nn.functional.interpolate(x, (x.shape[2], target_F), mode="bicubic", align_corners=True)  
        x = x.permute(0,1,3,2).contiguous() # B C F T
        x = x[:,:,:,cur_pos:cur_pos + self.spec_size]
        x = x.repeat(repeats = (1,1,4,1))
        return x

    def forward(self, x: torch.Tensor, mixup_lambda = None, infer_mode = False):# out_feat_keys: List[str] = None):
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)   
        if infer_mode:
            # in infer mode. we need to handle different length audio input
            frame_num = x.shape[2]
            target_T = int(self.spec_size * self.freq_ratio)
            repeat_ratio = math.floor(target_T / frame_num)
            x = x.repeat(repeats=(1,1,repeat_ratio,1))
            x = self.reshape_wav2img(x)
            output_dict = self.forward_features(x)
        else:
            x = self.reshape_wav2img(x)
            output_dict = self.forward_features(x)
        # x = x)
        # from pprint import pprint 
        # print("Output dict")
        # for (k,v) in output_dict.items():
        #     print(f"{k} shape", v.shape)
        return output_dict['clipwise_output']


from dataclasses import dataclass
@dataclass 
class HTSATConfig:

    loss_type = "clip_ce" # 
    # AudioSet & SCV2: "clip_bce" |  ESC-50: "clip_ce"     
    esc_fold = 0 # just for esc dataset, select the fold you need for evaluation and (+1) validation
    debug = False

    random_seed = 970131 # 19970318 970131 12412 127777 1009 34047
    # batch_size = 64 # batch size per GPU x GPU number , default is 32 x 4 = 128
    # learning_rate = 1e-3 # 1e-4 also workable 
    # max_epoch = 100
    # num_workers = 3
    token_label_range = [0.2,0.6]
    enable_time_shift = False # shift time
    enable_label_enhance = False # enhance hierarchical label
    enable_repeat_mode = False # repeat the spectrogram / reshape the spectrogram
    # for model's design
    # for signal processing
    sample_rate = 16000 # 16000 for scv2, 32000 for audioset and esc-50
    clip_samples = sample_rate * 10 # audio_set 10-sec clip
    window_size = 1024
    hop_size = 160 # 160 for scv2, 320 for audioset and esc-50
    mel_bins = 128 
    fmin = 50
    fmax = 14000
    shift_max = int(clip_samples * 0.5)
    # for data collection
    classes_num = 1 # esc: 50 | audioset: 527 | scv2: 35
    patch_size = (25, 4) # deprecated
    crop_size = None # int(clip_samples * 0.5) deprecated
    # for htsat hyperparamater
    htsat_window_size = 8
    htsat_spec_size =  256
    htsat_patch_size = 4 
    htsat_stride = (4, 4)
    htsat_num_head = [4,8,16,32]
    htsat_dim = 96 
    htsat_depth = [2,2,6,2]
    swin_pretrain_path = None
    # "/home/Research/model_backup/pretrain/swin_tiny_c24_patch4_window8_256.pth"
    # Some Deprecated Optimization in the model design, check the model code for details
    htsat_attn_heatmap = False
    htsat_hier_output = False 
    htsat_use_max = False
    # for ensemble test 
    ensemble_checkpoints = []
    ensemble_strides = []

In [5]:
import torch
import torch.nn as  nn
import torch.nn.functional as F
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from nnAudio import features
import torchaudio 


from wwv.data import AudioDataModule
from wwv.config import DataPaths
import torchaudio 
cfg = Config(params)
data_path = DataPaths(cfg.path['data_dir'], cfg.model_name, cfg.path['model_dir'])
data_module = AudioDataModule(data_path.root_data_dir + "/train.csv",
                              data_path.root_data_dir + "/val.csv",
                              data_path.root_data_dir + "/test.csv",
                              cfg=cfg)
                              
train_loader =  data_module.train_dataloader()
val_loader =  data_module.val_dataloader()
test_loader =  data_module.test_dataloader()

x = next(iter(train_loader))

config = HTSATConfig()

sed_model = HTSAT_Swin_Transformer(
    spec_size=config.htsat_spec_size,
    patch_size=config.htsat_patch_size,
    in_chans=1,
    num_classes=config.classes_num,
    window_size=config.htsat_window_size,
    config = config,
    depths = config.htsat_depth,
    embed_dim = config.htsat_dim,
    patch_stride=config.htsat_stride,
    num_heads=config.htsat_num_head
)


## Training

In [ ]:
x['x'].shape

In [6]:
import torch 
import torch.nn.functional as F 
from wwv.architecture import Architecture
from wwv.eval import Metric
from wwv.data import AudioDataModule
from wwv.config import  DataPaths
import statistics
# data_path = DataPaths(cfg.path['data_dir'], cfg.model_name, cfg.path['model_dir'])
cfg = Config(params)
# model = Architecture(cfg, training=True)
# model.extractor(torch.randn((1,48000))) # (torch.randn((1,48000)))
root = "/home/akinwilson/Code/pytorch/dataset/keywords"
# data_module = AudioDataModule(data_path.root_data_dir + "/train.csv",
#                               data_path.root_data_dir + "/val.csv",
#                               data_path.root_data_dir + "/test.csv",
#                                cfg)
# # model.processing_layer[3](x)
# train_loader=  data_module.train_dataloader()
# val_loader=  data_module.val_dataloader()
# channel, n_mels, time  = [1, 128, 114]

In [7]:
import torch
import torch.nn as  nn
import torch.nn.functional as F
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from nnAudio import features
import torchaudio 

from wwv.data import AudioDataModule
from wwv.config import DataPaths
import torchaudio 
cfg = Config(params)
data_path = DataPaths(cfg.path['data_dir'], cfg.model_name, cfg.path['model_dir'])
data_module = AudioDataModule(data_path.root_data_dir + "/train.csv",
                              data_path.root_data_dir + "/val.csv",
                              data_path.root_data_dir + "/test.csv",
                              cfg=cfg)
                              
train_loader =  data_module.train_dataloader()
val_loader =  data_module.val_dataloader()
test_loader =  data_module.test_dataloader()

x = next(iter(train_loader))

# x['x'].shape
# channel,  time, n_mels = [1, 241, 128]

In [8]:
from pytorch_lightning import Trainer
import pytorch_lightning as pl 
import torch.nn.functional as F 
from wwv.architecture import ResNet, Predictor, Bottleneck
from wwv.eval import Metric
from pytorch_lightning.loggers import TensorBoardLogger
from wwv.util import OnnxExporter
import bisect 
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,LearningRateMonitor, ModelPruning
from wwv.data import AudioDataModule
import torch 
from torch.optim.lr_scheduler import ReduceLROnPlateau


data_path = DataPaths(cfg.path['data_dir'], cfg.model_name, cfg.path['model_dir'])

cfg = Config(params)
# model = Architecture(cfg, training=True)
# model.extractor(torch.randn((1,48000))) # (torch.randn((1,48000)))
# model = Architecture(cfg, True)
data_module = AudioDataModule(data_path.root_data_dir + "/train.csv",
                              data_path.root_data_dir + "/val.csv",
                              data_path.root_data_dir + "/test.csv",
                              cfg=cfg)
                              
train_loader =  data_module.train_dataloader()
val_loader =  data_module.val_dataloader()
test_loader =  data_module.test_dataloader()
# model.processing_layer[3](x)


class Routine(pl.LightningModule):

    def __init__(self, model, cfg):
        super().__init__()
        self.model = model
        self.metric = Metric
        self.cfg = cfg
        self.lr = 1e-3


    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x = batch['x']
        y = batch['y']
        y_hat = self.model(x)
        y_hat = y_hat.squeeze()
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_hat = (F.sigmoid(y_hat) > 0.5).float()

        metrics = self.metric(y_hat, y, self.cfg)()
        return {"loss":loss, "train_ttr": metrics.ttr, "train_ftr": metrics.ftr, "train_acc": metrics.acc}


    def training_epoch_end(self, training_step_outputs):
        results = {
            "loss": torch.tensor([x['loss'].float().item() for x in training_step_outputs]).mean(),
            "ttr": torch.tensor([x['train_ttr'].float().mean().item() for x in training_step_outputs]).mean(),
            "ftr": torch.tensor([x['train_ftr'].float().mean().item() for x in training_step_outputs]).mean(),
            "acc": torch.tensor([x['train_acc'].float().mean().item() for x in training_step_outputs]).mean()
            }
        # self.log(f"LR",self.lr, on_epoch=True, prog_bar=True, logger=True)
        for (k,v) in results.items():
            self.log(f"train_{k}", v, on_epoch=True, prog_bar=True, logger=True)    


    def validation_step(self, batch, batch_idx):
        x = batch['x']
        y = batch['y']
        y_hat = self.model(x)
        # (batch, num_classes)
        y_hat = y_hat.squeeze()
        # (batch,)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        pred = F.sigmoid(y_hat)
        y_hat = (pred > 0.5).float()
        metrics = self.metric(y_hat, y)()
        return {"val_loss": loss, "val_ttr": metrics.ttr, "val_ftr": metrics.ftr, "val_acc": metrics.acc}


    def validation_epoch_end(self, validation_step_outputs):
        results = {
            "loss": torch.tensor([x['val_loss'].float().mean().item() for x in validation_step_outputs]).mean(),
            "ttr": torch.tensor([x['val_ttr'].float().mean().item() for x in validation_step_outputs]).mean(),
            "ftr": torch.tensor([x['val_ftr'].float().mean().item() for x in validation_step_outputs]).mean(),
            "acc": torch.tensor([x['val_acc'].float().mean().item() for x in validation_step_outputs]).mean()
            }
        for (k,v) in results.items():
            self.log(f"val_{k}", v, on_epoch=True, prog_bar=True, logger=True)    


    def test_step(self, batch, batch_idx):
        x = batch['x']
        y = batch['y']
        y_hat = self.model(x)
        # (batch, num_classes)
        y_hat = y_hat.squeeze()
        # (batch,)
        pred = F.sigmoid(y_hat)
        # (batch_probabilities,)
        y_hat = (pred > 0.5).float()
        # (batch_labels,)
        metrics = self.metric(y_hat, y)()
        return {"test_ttr": metrics.ttr, "test_ftr": metrics.ftr, "test_acc": metrics.acc}


    def test_epoch_end(self, test_step_outputs):
        results = {
            "ttr": torch.tensor([x['test_ttr'].float().mean().item() for x in test_step_outputs]).mean(),
            "ftr": torch.tensor([x['test_ftr'].float().mean().item() for x in test_step_outputs]).mean(),
            "acc": torch.tensor([x['test_acc'].float().mean().item() for x in test_step_outputs]).mean()
            }

        for (k,v) in results.items():
            self.log(f"test_{k}", v, on_epoch=True, prog_bar=True, logger=True)    


    def configure_optimizers(self):
        
        # optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        optimizer = torch.optim.AdamW(
            filter(lambda p: p.requires_grad, self.parameters()),
            lr = self.lr, 
            betas = (0.9, 0.999), eps = 1e-08, weight_decay = 0.05, 
        )

        # scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)

        def lr_warm_up(epoch):
            if epoch < 3:
                # warm up lr
                lr_scale = self.cfg.lr_rates[epoch]
            else:
                # warmup schedule
                lr_pos = int(-1 - bisect.bisect_left(self.cfg.lr_scheduler_epoch, epoch))
                if lr_pos < -3:
                    lr_scale = max(self.cfg.lr_rates[0] * (0.98 ** epoch), 0.03 )
                else:
                    lr_scale = self.cfg.lr_rates[lr_pos]
            return lr_scale


        scheduler = torch.optim.lr_scheduler.LambdaLR(
            optimizer,
            lr_lambda=lr_warm_up
        )
        
        return  {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"} 



lr_monitor = LearningRateMonitor(logging_interval='epoch')
early_stopping = EarlyStopping(mode="min", monitor='val_loss', patience=25)
checkpoint_callback = ModelCheckpoint(monitor="val_loss",
                                        dirpath=data_path.model_dir,
                                        save_top_k=1,
                                        mode="min",
                                        filename='{epoch}-{val_loss:.2f}-{val_acc:.2f}-{val_ttr:.2f}-{val_ftr:.2f}')




# # model = ResNet(block=Bottleneck, num_blocks=[8, 8, 36, 3], cfg=cfg)
# model = DeepSpeech(40)

config = HTSATConfig()

model = HTSAT_Swin_Transformer(
    spec_size=config.htsat_spec_size,
    patch_size=config.htsat_patch_size,
    in_chans=1,
    num_classes=config.classes_num,
    window_size=config.htsat_window_size,
    config = config,
    depths = config.htsat_depth,
    embed_dim = config.htsat_dim,
    patch_stride=config.htsat_stride,
    num_heads=config.htsat_num_head
)

callbacks = [checkpoint_callback, lr_monitor, early_stopping]

logger = TensorBoardLogger(save_dir=data_path.model_dir, version=1, name="lightning_logs")

trainer = Trainer(accelerator="gpu",
                  devices=3,
                  strategy='dp',
                  logger = logger, 
                  default_root_dir=data_path.model_dir,
                  callbacks=callbacks)
trainer.fit(Routine(model, cfg), train_dataloaders=train_loader, val_dataloaders=val_loader)

trainer.test(dataloaders=test_loader)


# from wwv.util import OnnxExporter
# model = trainer.model.module.module.model
# predictor = Predictor(model)
# OnnxExporter( model=predictor,
#              cfg=cfg, 
#              output_dir=data_path.model_dir)()

#####################################################################################################################
#                                            
#####################################################################################################################
# if isinstance(trainer.model, torch.nn.DataParallel):
#     print("test")
#     model = trainer.model
#####################################################################################################################
# reload best 
#####################################################################################################################
# automatically auto-loads the best weights from the previous run 
#####################################################################################################################

Created a temporary directory at /tmp/tmpbv_6s4ka
Created a temporary directory at /tmp/tmpbv_6s4ka
Created a temporary directory at /tmp/tmpbv_6s4ka
Created a temporary directory at /tmp/tmpbv_6s4ka
Writing /tmp/tmpbv_6s4ka/_remote_module_non_scriptable.py
Writing /tmp/tmpbv_6s4ka/_remote_module_non_scriptable.py
Writing /tmp/tmpbv_6s4ka/_remote_module_non_scriptable.py
Writing /tmp/tmpbv_6s4ka/_remote_module_non_scriptable.py
GPU available: True (cuda), used: True
GPU available: True (cuda), used: True
GPU available: True (cuda), used: True
GPU available: True (cuda), used: True
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
IPU available: False, us

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([10, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([10, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:03<00:03,  3.92s/it]forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([10, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([10, 64, 768])
Epoch 0:   0%|          | 0/27 [00:00<?, ?it/s]       

Restoring states from the checkpoint path at /home/akinwilson/Code/pytorch/output/model/HSTAT/epoch=10-val_loss=0.47-val_acc=0.83-val_ttr=0.00-val_ftr=0.00.ckpt
Restoring states from the checkpoint path at /home/akinwilson/Code/pytorch/output/model/HSTAT/epoch=10-val_loss=0.47-val_acc=0.83-val_ttr=0.00-val_ftr=0.00.ckpt
Restoring states from the checkpoint path at /home/akinwilson/Code/pytorch/output/model/HSTAT/epoch=10-val_loss=0.47-val_acc=0.83-val_ttr=0.00-val_ftr=0.00.ckpt
Restoring states from the checkpoint path at /home/akinwilson/Code/pytorch/output/model/HSTAT/epoch=10-val_loss=0.47-val_acc=0.83-val_ttr=0.00-val_ftr=0.00.ckpt
Restoring states from the checkpoint path at /home/akinwilson/Code/pytorch/output/model/HSTAT/epoch=10-val_loss=0.47-val_acc=0.83-val_ttr=0.00-val_ftr=0.00.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
LOC

Testing DataLoader 0:   0%|          | 0/3 [00:00<?, ?it/s]forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([10, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([10, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
Testing DataLoader 0:  33%|███▎      | 1/3 [00:00<00:00, 11.82it/s]forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([10, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([10, 64, 768])
forward_features[x, attn = layer(x)] [out]: torch.Size([11, 64, 768])
forward_features[self.norm(x)] [out]: torch.Size([11, 64, 768])
Testing DataLoader 0:  67%|██████▋   | 2/3 [00:00<00:00, 10.39it/s]for

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7414141297340393     │
│         test_ftr          │            0.0            │
│         test_ttr          │            0.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_ttr': 0.0, 'test_ftr': 0.0, 'test_acc': 0.7414141297340393}]

In [ ]:

class FullyConnected(torch.nn.Module):
    """
    Args:
        n_feature: Number of input features
        n_hidden: Internal hidden unit size.
    """

    def __init__(self, n_feature: int, n_hidden: int, dropout: float, relu_max_clip: int = 20) -> None:
        super(FullyConnected, self).__init__()
        self.fc = torch.nn.Linear(n_feature, n_hidden, bias=True)
        self.relu_max_clip = relu_max_clip
        self.dropout = dropout

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.hardtanh(x, 0, self.relu_max_clip)
        if self.dropout:
            x = torch.nn.functional.dropout(x, self.dropout, self.training)
        return x


class DeepSpeech(torch.nn.Module):
    """
    DeepSpeech model architecture from *Deep Speech: Scaling up end-to-end speech recognition*
    [:footcite:`hannun2014deep`].

    Args:
        n_feature: Number of input features
        n_hidden: Internal hidden unit size.
        n_class: Number of output classes
    """

    def __init__(
        self,
        n_feature: int=40,
        n_hidden: int = 4024,
        n_class: int = 1,
        dropout: float = 0.0,
    ) -> None:
        super().__init__()
        self.n_hidden = n_hidden
        self.fc1 = FullyConnected(n_feature, n_hidden, dropout)
        self.fc2 = FullyConnected(n_hidden, n_hidden, dropout)
        self.fc3 = FullyConnected(n_hidden, n_hidden, dropout)
        self.bi_rnn = torch.nn.RNN(n_hidden, n_hidden, num_layers=1, nonlinearity="relu", bidirectional=True)
        self.fc4 = FullyConnected(n_hidden, n_hidden, dropout)
        # self.flat = torch.nn.Flatten()

        self.out = torch.nn.Linear(241*n_hidden, n_class)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): Tensor of dimension (batch, channel, time, feature).
        Returns:
            Tensor: Predictor tensor of dimension (batch, time, class).
        """
# 241, 512
        # n_mfcc:40, timstep:241 (1,40, 241)
        # N x C x F x T 
        x = x.transpose(2,3)
        # N x C x T x F
        x = self.fc1(x)
        # N x C x T x H
        x = self.fc2(x)
        # N x C x T x H
        x = self.fc3(x)
        # N x C x T x H
        x = x.squeeze(1)
        # N x T x H
        x = x.transpose(0, 1)
        # T x N x H
        x, _ = self.bi_rnn(x)
        # The fifth (non-recurrent) layer takes both the forward and backward units as inputs
        x = x[:, :, : self.n_hidden] + x[:, :, self.n_hidden :]
        # T x N x H
        x = self.fc4(x)
        # T x N x H
        x = x.permute(1, 0, 2)
        # N x T x n_class
        # print(f"x = x.permute(1, 0, 2) {x.shape}")
        # print(f"{}")
        x = x.reshape(x.shape[0], -1)
        # print(f"x = x.reshape(x.shape[0], -1) {x.shape}")
        # N x (T x n_class)
        x = self.out(x)
        # N x n_class
        return x


import torch.nn as nn

class M5(nn.Module):
    def __init__(self, n_input=1, n_output=1, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        print(f"x = self.conv1(x) {self.conv1(x)}") 
        
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return x


x = torch.randn((1,48000))
M5()(x)

In [ ]:
class ProcessingLayer(nn.Module):

    def __init__(self,cfg):
        super().__init__()
        self.cfg =cfg
        layers = []
        kwargs = cfg.audio_feature_param[cfg.audio_feature]
        if cfg.audio_feature == "spectrogram":
            layers.append(features.MelSpectrogram(**kwargs))
            # layers.append(T.Resize(224)) # size expected by 2D ResNet 
        elif cfg.audio_feature == "mfcc":
            layers.append(features.MFCC(**kwargs))
            # layers.append(T.Resize(224)) # size expected by 2D ResNet

        # resize inputs
        # layers.append(transforms.RandomResizedCrop(224))
        self.net = torch.nn.Sequential(*layers)
        # logger.info(f"{'-'*20}> Features to be extracted: {cfg.audio_feature}")
        # logger.info(f"{'-'*20}> Feature dimensions: {cfg.processing_output_shape}")

    def forward(self, x:torch.tensor) -> torch.tensor:
        x_out = self.net(x)
        # if self.cfg.verbose:
        #     logger.info(f"ProcessingLayer().foward() [in]: {x.shape}")
        #     logger.info(f"ProcessingLayer().foward() [out]: {x_out.shape}")
        return x_out 


class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""

    def __init__(self, n_feats):
        super().__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous()  # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous()  # (batch, channel, feature, time)


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
    except with layer norm instead of batch norm
    """

    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super().__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel // 2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel // 2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x  # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):
    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super().__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=batch_first,
            bidirectional=True,
        )
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x

from math import prod

class DeepSpeech(nn.Module):

    def __init__(self, n_cnn_layers=20, n_rnn_layers=1, rnn_dim=1096, stride=2, dropout=0.1,cfg=cfg, **kwargs):
        super().__init__()
        self.cfg= cfg 
        #  =n_mfcc:40, timstep () 40, 241
        
        def ceildiv(a, b):
            return -(a // -b)

        in_feat_dim = (121//2, 20//2) #  self.cfg.processing_output_shape
        # self.processing_layer = ProcessingLayer(cfg)
        # n_feats =  (121 * 20) // 2
    
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3 // 2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(
            *[ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=in_feat_dim) for _ in range(n_cnn_layers)]
        )
        
        self.fully_connected = nn.Linear(121, rnn_dim)
        self.birnn_layers = nn.Sequential(
            *[
                BidirectionalGRU(
                    rnn_dim=rnn_dim if i == 0 else rnn_dim * 2,
                    hidden_size=rnn_dim,
                    dropout=dropout,
                    batch_first=i == 0,
                )
                for i in range(n_rnn_layers)
            ]
        )
        self.classifier = nn.Sequential(
            nn.Linear(1402880, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, 1),
        )

    def forward(self, x):
        print(f"Shape b cnn {x.shape}")
        # x = self.processing_layer(x) # (batch, mfcc, timestep)
        # print(f"Shape a process {x.shape}")
        # x = x.unsqueeze(1) # (batch, channel,  mfcc, timestep)
        # print(f"Shape a transpose {x.shape}")
        x = self.cnn(x)
        print(f"Shape a cnn {x.shape}")
        x = self.rescnn_layers(x)
        print(f"Shape a rescnn {x.shape}")
        
        # # print(f"after view {x.shape}")
        # x = x.transpose(1, 2)  # (batch, time, feature)

        x = self.fully_connected(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        print(f"Shape a fully_connected {x.shape}")
        x = self.birnn_layers(x)
        print(f"Shape a birnn_layers {x.shape}")
        # print(f"after birnn_layers {x.shape}")
        x =  self.flatten(x)
        x = self.classifier(x)
        return x



In [ ]:
import torch 
import torch.nn as nn 
n_feats = (1,121, 20) #  self.cfg.processing_output_shape
# self.processing_layer = ProcessingLayer(cfg)
# n_feats =  (121 * 20) // 2


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
    except with layer norm instead of batch norm
    """

    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super().__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel // 2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel // 2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x  # (batch, channel, feature, time)

xin = torch.randn(n_feats)
stride=2
cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3 // 2)
# (61, 10)
def ceildiv(a, b):
    return -(a // -b)

n_feats = (ceildiv(121,2), ceildiv(20,2) )


print(n_feats)
rescnn_layers = nn.Sequential(
            *[ResidualCNN(32, 32, kernel=3, stride=1, dropout=0.1, n_feats=n_feats) for _ in range(2)]
        )
        
# .shape

rescnn_layers(cnn(xin))

In [ ]:
from transformers import Wav2Vec2FeatureExtractor, SEWDForSequenceClassification
from datasets import load_dataset
import torch

# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate
class SEW(nn.Module):

    def __init__(self):
        super().__init__()
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("anton-l/sew-d-mid-400k-ft-keyword-spotting")
        self.model = SEWDForSequenceClassification.from_pretrained("anton-l/sew-d-mid-400k-ft-keyword-spotting")


    def forward(self, x):
        # x = input_dict['input_values']
        x_feats = self.feature_extractor(x, ampling_rate=sampling_rate, return_tensors="pt")
        x_feats.unsqueeze(1)
        logits = self.model(x)
        return logits 

dataset[0]["audio"]["array"]
sew  =SEW()
# # audio file is decoded on the fly

inputs = dataset[0]["audio"]["array"]
with torch.no_grad():
    logits = sew(torch.tensor(inputs))
print(logits)
# predicted_class_ids = torch.argmax(logits, dim=-1).item()
# predicted_label = model.config.id2label[predicted_class_ids]
# predicted_label

In [ ]:
trainer = Trainer()
# Run learning rate finder
model = ResNet(block=Bottleneck, num_blocks=[3, 8, 36, 3], cfg=cfg)
model = Routine(model, cfg)

lr_finder = trainer.tuner.lr_find(model)

# Results can be found in
lr_finder.results

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

# Pick point based on plot, or get suggestion
# new_lr = lr_finder.suggestion()

# # update hparams of the model
# model.hparams.lr = new_lr

In [ ]:

# TensorRT 
rt = {
  "error": False,
  "result": {
    "wake_word_probability": 0,
    "prediction": 0,
    "false_alarm_probability": 1,
    "decision_threshold": 0.5,
    "wwvm_version": "docker-env-model-version",
    "inference_time": 0.01810431480407715
  }
}

# with CPU
cpu = {
  "error": False,
  "result": {
    "wake_word_probability": 0,
    "prediction": 0,
    "false_alarm_probability": 1,
    "decision_threshold": 0.5,
    "wwvm_version": "docker-env-model-version",
    "inference_time": 0.08728623390197754
  }
}


# with cuda 
cuda = {
  "error": False,
  "result": {
    "wake_word_probability": 0,
    "prediction": 0,
    "false_alarm_probability": 1,
    "decision_threshold": 0.5,
    "wwvm_version": "docker-env-model-version",
    "inference_time": 0.022240400314331055
  }
}

def get_factor(d1,d2):
  return d1['result']['inference_time'] / d2['result']['inference_time']


print(f"Cuda {get_factor(cpu,cuda):.2f} faster than cpu")
print(f"TensorRT {get_factor(cpu,rt):.2f} faster than cpu")
print(f"TensorRT {get_factor(cuda,rt):.2f} faster than cuda")

In [ ]:
# import torch 
# PATH = "/home/akinwilson/Code/pytorch/output/model/epoch=27-val_loss=0.16-val_acc=0.97.ckpt"
# model.load_state_dict(torch.load(PATH), map_location=torch.device('cpu'))
# trainer.test(test_loader, ckpt_path='best')
from torch import tensor 
# ftrs = [x['train_ftr'].mean().item() for x in training_step_outputs]
# accs = [x['train_acc'].mean().item() for x in training_step_outputs]
# losses
# ttrs
# results = {"avg_loss": statistics.fmean([x['loss'].item() for x in training_step_outputs]),}
            # "avg_ttr": torch.stack([x['train_ttr'].mean().item() for x in training_step_outputs]).mean(),
            # "avg_ftr": torch.stack([x['train_ftr'].mean().item() for x in training_step_outputs]).mean(),
            # "avg_acc": torch.stack([x['train_acc'].mean().item() for x in training_step_outputs]).mean()}

In [ ]:
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

xs =torch.ones(64, 48000)
ys = torch.ones(64)

trainset = dataset(xs,ys)
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)
for b in trainloader:
  print(b[0].shape)